In [1]:
import numpy as np
import keras
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Flatten
from keras.layers.core import Dense, Dropout
from keras.callbacks import TensorBoard
# from keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt
import scipy.io as sio 
from sklearn.preprocessing import normalize
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
matfn = 'posture_5_data.mat'

data = sio.loadmat(matfn)  
x_train = data['x_train'].T
x_test = data['x_test'].T
y_train = data['y_train'].T
y_test = data['y_test'].T

print("x_train", x_train.shape)
print("x_test ", x_test.shape)
print("y_train", y_train.shape)
print("y_test ", y_test.shape)

#将数据集数据进行归一化，归一方法采用StandardScaler()
# # dataset Standardscalization
# scaler = StandardScaler()

# x_train = scaler.fit_transform(
#     x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 200, 16)

# x_test = scaler.transform(
#     x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 200, 16)

x_train = x_train.astype(np.float32).reshape(-1, 200, 16)
x_test = x_test.astype(np.float32).reshape(-1, 200, 16)


print("x_train", x_train.shape)
print("x_test ", x_test.shape)
print("y_train", y_train.shape)
print("y_test ", y_test.shape)

print('************ label to on-hot ***************')
y_train= np.transpose(y_train)
y_test = np.transpose(y_test)

y_train = np_utils.to_categorical(y_train, num_classes=5)
y_test = np_utils.to_categorical(y_test, num_classes=5)

print("y_train", y_train)
print("y_test ", y_test)


x_train (3200, 1000)
x_test  (3200, 250)
y_train (1, 1000)
y_test  (1, 250)
x_train (1000, 200, 16)
x_test  (250, 200, 16)
y_train (1, 1000)
y_test  (1, 250)
************ label to on-hot ***************
y_train [[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]]
y_test  [[0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]]


In [3]:
# 此部分为CNN神经网络内部结构
model = Sequential()
#创建序列模型
model.add(Dense(128,kernel_initializer='random_uniform', bias_initializer='zeros'))
#将训练数据输入进行初始化
# Conv layer 1 output shape (1,50,32)16,200,32
# 添加卷积层1 各个参数什么意思百度 
#注释后面参数不对，不用管，懒得改，后面类似
model.add(Convolution1D(filters=16,kernel_size=3,strides=1, padding='same',))   # Padding method
model.add(Activation('relu'))
#Relu激活函数
# Pooling layer 1 (max pooling) output shape (1, 25, 32)
model.add(MaxPooling1D(pool_size=3,strides=3,padding='same',))    # Padding method
#添加池化层1，各个参数什么意思百度
# =====================================================
# Conv layer 2 output shape (1,25,64)
model.add(Convolution1D(32, 3, strides=1, padding='same'))
model.add(Activation('relu'))

# Pooling layer 2 (max pooling) output shape (1, 25, 32)
model.add(MaxPooling1D(2, 2, 'same'))
model.add(Activation('relu'))
# =====================================================
# Conv layer 3 output shape (1，50，128)
model.add(Convolution1D(64, 3, strides=1, padding='same'))
model.add(Activation('relu'))

# Pooling layer 3 (max pooling) output shape (1, 25, 32)
model.add(MaxPooling1D(2, 2, 'same'))

# =====================================================
# Conv layer 4 output shape (1，50，128)
model.add(Convolution1D(128, 3, strides=1, padding='same'))
model.add(Activation('relu'))

# Pooling layer 4 (max pooling) output shape (1, 25, 32)
model.add(MaxPooling1D(2, 2, 'same'))

model.add(Dropout(0.5))
# 添加dropout层，训练时丢掉一部分数据，防止训练过拟合，失去泛化能力

# Fully connected layer 1 input shape (1*25*128) 
model.add(Flatten())
# 将网络铺平,以连通到后面的全连接层

model.add(Dense(2000, activation='relu'))
#添加512通道的全连接层1 激活函数为Relu

model.add(Dropout(0.5))
# 添加dropout层，训练时丢掉一部分数据，防止训练过拟合，失去泛化能力

# Fully connected layer 2 to shape (20) for 10 classes
model.add(Dense(5))
model.add(Activation('softmax'))
# 添加20通道的全连接层，用于将结果分类为20中不同的姿势
# 采用Softmax激活函数

# Another way to define your optimizer
adam = Adam(lr=1e-2)
#训练优化器采用Adam，学习速率是0.0001

# We add metrics to get more results you want to see
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#损失函数定义为交叉熵，衡量标准定义为准确度

2022-03-26 00:21:14.967705: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/fangaoyige/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [4]:
print('Training ------------')

# Tensorboard = TensorBoard(log_dir="./history") # Monitor in real time
#调用Tensorboard进行实时监控，不会用建议注释掉这一行

history = model.fit(x_train, y_train, epochs=200, batch_size=128 , validation_data=(x_test, y_test), verbose=1)

score = model.evaluate(x_test, y_test, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
loss, accuracy = model.evaluate(x_test, y_test)

print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)





Training ------------
Epoch 1/200
8/8 [==============================] - 6s 435ms/step - loss: 317.8624 - accuracy: 0.2080 - val_loss: 3.1868 - val_accuracy: 0.1840
Epoch 2/200
8/8 [==============================] - 2s 294ms/step - loss: 1.6172 - accuracy: 0.1850 - val_loss: 3.4006 - val_accuracy: 0.2160
Epoch 3/200
8/8 [==============================] - 2s 217ms/step - loss: 1.6092 - accuracy: 0.2050 - val_loss: 4.1042 - val_accuracy: 0.1800
Epoch 4/200
8/8 [==============================] - 2s 209ms/step - loss: 1.6112 - accuracy: 0.2050 - val_loss: 1.6141 - val_accuracy: 0.1680
Epoch 5/200
8/8 [==============================] - 2s 199ms/step - loss: 1.6096 - accuracy: 0.2180 - val_loss: 1.6152 - val_accuracy: 0.1680
Epoch 6/200
8/8 [==============================] - 2s 219ms/step - loss: 1.6110 - accuracy: 0.1890 - val_loss: 1.6131 - val_accuracy: 0.1760
Epoch 7/200
8/8 [==============================] - 2s 235ms/step - loss: 1.6110 - accuracy: 0.1880 - val_loss: 1.6123 - val_accura

KeyboardInterrupt: 

In [ ]:
y_pre = np.argmax(model.predict(x_test),axis=1)
print(y_pre)

# 绘制训练 & 验证的准确率值
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# 绘制训练 & 验证的损失值
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
matfn = 'posture_5_data.mat'
data = sio.loadmat(matfn)  
y_test = data['y_test']

y_test.resize(250,)
print(type(y_test))
print(np.array(y_test).shape)
print(y_test)

print(np.array(y_pre).shape)
print(type(y_pre))
print(y_pre)

In [ ]:
con_mat = confusion_matrix(y_test, y_pre)

con_mat_norm = con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis]     # 归一化
con_mat_norm = np.around(con_mat_norm, decimals=2)

# === plot ===
plt.figure(figsize=(5, 5))
sns.heatmap(con_mat_norm, annot=True, cmap='Blues')

plt.ylim(0, 5)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()